# Nye aviser

In [1]:
import dhlab.nbtext as nb
import dhlab.graph_networkx_louvain as gnl
import dhlab.nbtokenizer as tok
from dhlab.module_update import css, update, code_toggle

In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
%matplotlib inline

In [24]:
def get_df(frases, title='aftenposten', media='aviser'):
    import requests
    querystring = " + ".join(['"'+frase+'"' for frase in frases])
    query = {
        'q':querystring,
        'size':1,
        'aggs':'year',
        'filter':['mediatype:{mt}'.format(mt=media),'title:{title}'.format(title=title)]
    }
    r = requests.get("https://api.nb.no/catalog/v1/items", params = query)
    aggs = r.json()['_embedded']['aggregations'][0]['buckets']
    return {x['key']:x['count'] for x in aggs}

def get_json(frases, mediatype='aviser', title='*'):
    import requests
    querystring = " + ".join(['"'+frase+'"' for frase in frases])
    query = {
        'q':querystring,
        'size':1,
        'snippets':mediatype,
        'aggs':'year',
        
       'filter':['mediatype:{mt}'.format(mt=mediatype),'title:{title}'.format(title=title)]
        'searchType':'FULL_TEXT_SEARCH'
        #'filter':
    }
    r = requests.get("https://api.nb.no/catalog/v1/items", params = query)
    aggs = r.json()
    return aggs

def get_data(frase, media='aviser', title='jazznytt'):
    import requests
    query = {
        'q':'"'+frase+'""',
        'size':1,
        'aggs':'year',
        'filter':['mediatype:{mt}'.format(mt=media),'title:{title}'.format(title=title)]
        
    }
    r = requests.get("https://api.nb.no/catalog/v1/items", params = query)
    return r.json()

def get_data_and(frases, title='aftenposten', media='aviser'):
    import requests
    querystring = " + ".join(['"'+frase+'"' for frase in frases])
    print(querystring)
    query = {
        'q':querystring,
        'size':1,
        'aggs':'year',
        'filter':['mediatype:{mt}'.format(mt=media), 'title:{title}'.format(title=title)],
        #'filter':'title:{title}'.format(title=title)
    }
    r = requests.get("https://api.nb.no/catalog/v1/items", params = query)
    return r.json()

def get_df_pd(frase, media='bøker'):
    import pandas as pd
    return pd.DataFrame.from_dict(get_df(frase, media=media ), orient='index').sort_index()

In [4]:
def phrase_plots(phrase_sets, title='aftenposten', fra = 1960, til = 2020, step=5, rot=0, colours = ['r', 'b','g','y','m','c']):
    df_all = []
    for f in phrase_sets:
        df_all.append(nb.frame(get_df(f, title= title), ', '.join(f)))
    df = pd.concat(df_all, sort=False)
    df.index = df.index.astype(int)
    df = df.sort_index()
    df['bins'] = pd.cut(df.index, range(fra, til, step), precision=0)
    df.groupby('bins').sum().plot(kind='bar', color=colours, figsize=(15,5), rot=rot)
    return

def phrase_plots_anno(phrase_sets, title='aftenposten', fra = 1960, til = 2020, rot=0, colours = ['r', 'b','g']):
    df_all = []
    for f in phrase_sets:
        df_all.append(nb.frame(get_df(f, title= title), ', '.join(f)))
    df = pd.concat(df_all, sort=False)
    df.index = df.index.astype(int)
    df = df.sort_index()
    #df['bins'] = pd.cut(df.index, range(fra, til, step), precision=0)
    df.plot(kind='bar', figsize=(15,5), rot=rot, color=colours)
    return

def graph_from_df(df, threshold = 100):
    edges =  []
    normalizer = {(x, y): df.stack()[(x,x)]*df.stack()[(y,y)] for (x,y) in df.stack().index}
    for (x, y) in df.stack().index:
        if x != y:
            if df.stack()[(x,y)] > threshold:
                edges.append([x,y,df.stack()[(x,y)]/normalizer[(x,y)]])
    G = nx.Graph()
    G.add_weighted_edges_from(edges)
    return G

In [27]:
def super_search(title, number=50, page=0, mediatype='aviser'):
    """Søk etter term og få ut json"""
    number = min(number, 50)
    r = requests.get(
        "https://api.nb.no:443/catalog/v1/items", 
         params = { 
             'filter':['mediatype:{mediatype}'.format(mediatype=mediatype),
                      'title:{title}'.format(title=title)],
             'page':page, 
             'size':number
         }
    )
    return r.json()

In [110]:
def term_urn_search(term, number=50, page=0, mediatype='aviser'):
    """Søk etter term og få ut json"""
    number = min(number, 50)
    r = requests.get(
        "https://api.nb.no:443/catalog/v1/search", 
         params = { 
             'q':term,
             'sort':'date,desc',
             'date_from':'20180101',
             'filter':'mediatype:{mediatype}'.format(mediatype=mediatype),
             'page':page, 
             'size':number,
             'random':'true'
         }
    )
    res = r.json()
    tot = res['totalElements']
    result = res['_embedded']['mediaTypeResults'][0]['result']['_embedded']['items']
    urns = [r['metadata']['identifiers']['urn'] for r in result]
    return urns

In [110]:
def term_urn_search(term, number=50, page=0, mediatype='aviser'):
    """Søk etter term og få ut json"""
    number = min(number, 50)
    r = requests.get(
        "https://api.nb.no:443/catalog/v1/search", 
         params = { 
             'q':term,
             'sort':'date,desc',
             'date_from':'20180101',
             'filter':'mediatype:{mediatype}'.format(mediatype=mediatype),
             'page':page, 
             'size':number,
             'random':'true'
         }
    )
    res = r.json()
    tot = res['totalElements']
    result = res['_embedded']['mediaTypeResults'][0]['result']['_embedded']['items']
    urns = [r['metadata']['identifiers']['urn'] for r in result]
    return urns

In [130]:
requests.get("https://api.nb.no/catalog/v1/search?q=corona&mediatype=aviser&fromDate=20200101&toDate=20200231").json()


{'totalElements': 88920,
 '_links': {'self': {'href': 'https://api.nb.no/catalog/v1/search?q=corona&mediatype=aviser&fromDate=20200101&toDate=20200231'}},
 '_embedded': {'mediaTypeResults': [{'mediaType': 'aviser',
    'result': {'_links': {'self': {'href': 'https://api.nb.no/catalog/v1/items?q=corona&filter=mediatype:aviser&page=0&size=10'},
      'next': {'href': 'https://api.nb.no/catalog/v1/items?q=corona&filter=mediatype:aviser&page=1&size=10'},
      'last': {'href': 'https://api.nb.no/catalog/v1/items?q=corona&filter=mediatype:aviser&page=50&size=10'}},
     'page': {'size': 10,
      'totalElements': 81472,
      'totalPages': 8148,
      'number': 0},
     '_embedded': {'items': [{'id': 'd19c2e812309a105eed8a4d10cf2ee99',
        '_links': {'self': {'href': 'https://api.nb.no:443/catalog/v1/items/d19c2e812309a105eed8a4d10cf2ee99'},
         'mods': {'href': 'https://api.nb.no:443/catalog/v1/metadata/d19c2e812309a105eed8a4d10cf2ee99/mods'},
         'dublincore': {'href': 'http

In [111]:
term_urn_search('corona', page=1)

['URN:NBN:no-nb_digavis_mossavis_null_null_19750626_99_143_1',
 'URN:NBN:no-nb_digavis_aftenposten_null_null_19390603_80_271_3',
 'URN:NBN:no-nb_digavis_hadeland_null_null_19890912_72_175_1',
 'URN:NBN:no-nb_digavis_nationen_null_null_19770127_59_22_1',
 'URN:NBN:no-nb_digavis_dagenbergen_null_null_19300220_12_43_1',
 'URN:NBN:no-nb_digavis_bergensavisen_null_null_19920228_66_58_1',
 'URN:NBN:no-nb_digavis_morgenbladet_null_null_19140208_96_71_1',
 'URN:NBN:no-nb_digavis_faedrelandsvennen_null_null_19391012_65_237_1',
 'URN:NBN:no-nb_digavis_opplandarbeiderblad_null_null_19940701_71_147_1',
 'URN:NBN:no-nb_digavis_stavangeraftenblad_null_null_19850709_93_154_1',
 'URN:NBN:no-nb_digavis_nationen_null_null_19250527_8_120_1',
 'URN:NBN:no-nb_digavis_norgeshandelsogsjoefartstidende_null_null_19830825_94_163_1',
 'URN:NBN:no-nb_digavis_tonsbergsblad_null_null_19750912_106_210_1',
 'URN:NBN:no-nb_digavis_norgeshandelsogsjoefartstidende_null_null_19830707_94_128_1',
 'URN:NBN:no-nb_digavis_ve

In [5]:
def get_konks(urn, phrase, window=1000, n = 1000):
    import requests
    querystring = '"'+ phrase +'"' 
    query = {
        'q':querystring,
        'fragments': n,
        'fragSize':window
       
    }
    r = requests.get("https://api.nb.no/catalog/v1/items/{urn}/contentfragments".format(urn=urn), params = query)
    res = r.json()
    results = []
    try:
        for x in res['contentFragments']:
            urn = x['pageid']
            hit = x['text']
            splits = hit.split('<em>')
            s2 = splits[1].split('</em>')
            before = splits[0]
            word = s2[0]
            after = s2[1]
            results.append({'urn': urn, 'before': before, 'word':word, 'after':after})
    except:
        True
    return results

In [6]:
r = get_konks("URN:NBN:no-nb_digibok_2011021403034", "pølse i handa", window=2000)
r

[{'urn': 'URN:NBN:no-nb_digibok_2011021403034_0009',
  'before': 'gjør vi jo alle pølser: En enslig mann på en bensinstasjon tre! ved middagstider, med ei ',
  'word': 'pølse i handa,',
  'after': ' bryter god spiseskikk. Pølsa blir da for mange et symbol på forfall. Vi har ikke som'}]

In [80]:
konks = []
for u in term_urn_search('corona', page=1):
    konks += get_konks(u, 'corona')
konks

[{'urn': 'URN:NBN:no-nb_digavis_finnmarkdagblad_null_null_19771108_65_259_1-1_016_null',
  'before': 'mod. .1^ -| CDFCIål ICTFII 4-dørs 74 mod. Opel Record, Oi tolALIo I til Toyota ',
  'word': 'Corona',
  'after': ' stasjon 72 mod. \\ 55 — -r., nun 4- dørs 70 mod. Volvo 145, BJARNE HAY'},
 {'urn': 'URN:NBN:no-nb_digavis_bergenstidende_null_null_19740607_107_128_1-1_012_null',
  'before': 'Tandberg k 147, 5 Ck 145) ~ Viking-Askim 0 119 Co 120) SKIPSAKSJER Beaulieu k 900 (s 1100) ',
  'word': 'Corona',
  'after': ' k 1400 Ck 1450) NAL k 285 (o 280) Bergenske s 105 Ck 90) Nordenfj. • ingen not.'},
 {'urn': 'URN:NBN:no-nb_digavis_bergenstidende_null_null_19740607_107_128_1-1_026_null',
  'before': 'VW v.v. 69 mod. VW pick up 69 mod. Avbet. Tlf. 25 40 00 Åsane Bruktbil. TOYOTA ',
  'word': 'Corona',
  'after': ' -67 selges gr reise. Pen. god stand. Kr. 8000. Tlf. 29 80 50 1. 218. e. kl. 16.00'},
 {'urn': 'URN:NBN:no-nb_digavis_romsdalsbudstikke_null_null_19820127_140_22_1-1_027_null',
  'b

In [66]:
nb.concordance(word='morsomt', corpus='avis', title='aftenposten', kind="json")

[{'after': '. Når Kristopher Schaus pølsestunt',
  'before': '; de synes medier ev',
  'urn': 'https://www.nb.no/items/URN:NBN:no-nb_digavis_aftenposten_null_null_20040204_145_58_2',
  'word': 'morsomt'},
 {'after': ". Kristoffer Nordvik Ti '",
  'before': 'det etter all sannsynlighet blir',
  'urn': 'https://www.nb.no/items/URN:NBN:no-nb_digavis_aftenposten_null_null_20040204_145_58_2',
  'word': 'morsomt'},
 {'after': 'kurs vil dere få kunnskap',
  'before': 'Gjennom gratis , lærerikt og',
  'urn': 'https://www.nb.no/items/URN:NBN:no-nb_digavis_aftenposten_null_null_20040204_145_58_2',
  'word': 'morsomt'},
 {'after': 'kurs vil dere få kunnskap',
  'before': 'Gjennom gratis , lærerikt og',
  'urn': 'https://www.nb.no/items/URN:NBN:no-nb_digavis_aftenposten_null_null_20040204_145_58_2',
  'word': 'morsomt'},
 {'after': ', innrømmer 18 - åringen',
  'before': ', men det er ikke',
  'urn': 'https://www.nb.no/items/URN:NBN:no-nb_digavis_aftenposten_null_null_20040204_145_58_2',
  'word':